In [1]:
from pylab import *
import numpy as np
from trackeddy.datastruct import *
%matplotlib inline
import gsw

In [2]:
eddydict=np.load('../data_bk/aviso_1993-1-3_pos.npy')

In [3]:
eddyd=eddydict.item()

In [4]:
new_dict_0={}
new_dict_1={}
new_dict_2={}
for key,item in eddyd.items():
    if int(item['time'])==0:
        new_dict_0[key]=item
    elif int(item['time'])==1:
        new_dict_1[key]=item
    elif int(item['time'])==2:
        new_dict_2[key]=item
    else:
        break

In [5]:
new_dict_0.keys(),new_dict_1.keys()

(dict_keys(['eddyn_0', 'eddyn_1', 'eddyn_2', 'eddyn_3', 'eddyn_4', 'eddyn_5', 'eddyn_6', 'eddyn_7', 'eddyn_8', 'eddyn_9', 'eddyn_10', 'eddyn_11', 'eddyn_12', 'eddyn_13', 'eddyn_14', 'eddyn_15', 'eddyn_16', 'eddyn_17', 'eddyn_18', 'eddyn_19', 'eddyn_20', 'eddyn_21', 'eddyn_22', 'eddyn_23', 'eddyn_24', 'eddyn_25', 'eddyn_26', 'eddyn_27', 'eddyn_28', 'eddyn_29', 'eddyn_30', 'eddyn_31', 'eddyn_32', 'eddyn_33', 'eddyn_34', 'eddyn_35', 'eddyn_36', 'eddyn_37', 'eddyn_38', 'eddyn_39', 'eddyn_40', 'eddyn_41', 'eddyn_42', 'eddyn_43', 'eddyn_44', 'eddyn_45', 'eddyn_46', 'eddyn_47', 'eddyn_48', 'eddyn_49', 'eddyn_50', 'eddyn_51', 'eddyn_52', 'eddyn_53', 'eddyn_54', 'eddyn_55', 'eddyn_56', 'eddyn_57', 'eddyn_58', 'eddyn_59', 'eddyn_60', 'eddyn_61', 'eddyn_62', 'eddyn_63', 'eddyn_64', 'eddyn_65', 'eddyn_66', 'eddyn_67', 'eddyn_68', 'eddyn_69', 'eddyn_70', 'eddyn_71', 'eddyn_72', 'eddyn_73', 'eddyn_74', 'eddyn_75', 'eddyn_76', 'eddyn_77', 'eddyn_78', 'eddyn_79', 'eddyn_80', 'eddyn_81', 'eddyn_82', 'e

In [6]:
class eddyTimeTrack():
    '''
    
    '''
    def __init__(self,eddies_at_t,eddies_at_t1,method='nearest',mode='offline'):
        self.method=method
        self.mode=mode
        self.tracked_dict={}
        if type(eddies_at_t)==dict:
            self.eddy_t=eddies_at_t
            self.eddy_t1=eddies_at_t1
            self.keys_at_t=list(self.eddy_t.keys())
            self.keys_at_t1=list(self.eddy_t1.keys())
            self.type='dict'
        elif type(eddies_at_t)==pd.Dataframe:
            pass
            self.type='pandas'
        else:
            raise TypeError('The input of this tracking method should be a dictionary or a pandas dataframe.')
    
    def track_eddies(self):
        if self.type=='dict':
            self.tracked_dict={}
            tracks2connect=self.persisting_eddies()
            for time_connected_eddies in tracks2connect:
                self.tracked_dict[time_connected_eddies[0]]={}
                for key_prop_eddies,values_prop_eddies_t in self.eddy_t[time_connected_eddies[0]].items():
                    if key_prop_eddies != 'timetracking' and key_prop_eddies!='neddy':
                        if type(values_prop_eddies_t)==np.ndarray:
                            self.tracked_dict[time_connected_eddies[0]][key_prop_eddies]=np.vstack((values_prop_eddies_t,self.eddy_t1[time_connected_eddies[1]][key_prop_eddies]))
                        else:
                            self.tracked_dict[time_connected_eddies[0]][key_prop_eddies]=[values_prop_eddies_t[0],self.eddy_t1[time_connected_eddies[1]][key_prop_eddies][0]]
                    else:
                        self.tracked_dict[time_connected_eddies[0]][key_prop_eddies]=values_prop_eddies_t
        else:
            pass
        
        if len(tracks2connect)*2+len(self.keys_at_t)+len(self.keys_at_t1)!=len(self.eddy_t.keys())+len(self.eddy_t1.keys()):
            raise ValueError('Leakage of eddies, look into the tracking algorithm.')
        else:
            self.append_non_connected()

        return self.tracked_dict
        
    
    def persisting_eddies(self):
        connect_tracks=[]
        non_repeated_eddy=[]
        #for key_t0 in range(len(time_0_key)):
        key_t0=0
        while len(self.keys_at_t)>key_t0:
            loc=self.get_loc(key_t0)
            if self.method=='nearest':
                
                locs_t=self.list_locs()
                near_eddy,distance_between_eddies=self.nearest_eddy(loc,locs_t)
                
                rossby_radius=rossbyR(loc[0],loc[1])

                if distance_between_eddies < rossby_radius and near_eddy not in non_repeated_eddy and self.eddy_t[self.keys_at_t[key_t0]]['timetracking']<6:
                        connect_tracks.append([self.keys_at_t[key_t0],self.keys_at_t1[near_eddy]])
                        non_repeated_eddy.append(near_eddy)
                        self.keys_at_t1[near_eddy]=None
                        self.keys_at_t[key_t0]=None
            elif self.method=='weighted':
                pass
            
            key_t0+=1
        self.keys_at_t=[ii for ii in self.keys_at_t if ii!=None]
        self.keys_at_t1=[ii for ii in self.keys_at_t1 if ii!=None]

        return connect_tracks
    
    def append_non_connected(self):
        for key, value in self.eddy_t.items(): 
            if key in self.keys_at_t:
                self.tracked_dict[key]=value
                self.tracked_dict[key]['timetracking']+=1
        count=0
        current_number_eddies=len(self.tracked_dict.keys())
        for key, value in self.eddy_t1.items():
            if key in self.keys_at_t1:
                count+=1
                self.tracked_dict['eddy_n{0}'.format(count+current_number_eddies)]=value
                self.tracked_dict['eddy_n{0}'.format(count+current_number_eddies)]['timetracking']+=1
    
    def nearest_eddy(self,loc,locs_t):
        near_eddy=abs(sum(locs_t-loc,axis=1)).argmin()
        distance_between_eddies=sw.dist(np.array([loc[1],locs_t[near_eddy][1]]),np.array([loc[0],locs_t[near_eddy][0]]))[0]*1000 #Distance in meters
        return near_eddy,distance_between_eddies
    
    def weighted_nearest_eddy(self,loc,loc_t,previous_orientation):
        near_eddy=abs(sum(locs_t-loc,axis=1)).argmin()
        distance_between_eddies,angle_trajectory=sw.dist(np.array([loc[1],locs_t[near_eddy][1]]),np.array([loc[0],locs_t[near_eddy][0]]))
        
        return near_eddy,distance_between_eddies*1000,angle_trajectory
    
    def get_loc(self,ii):
        if self.type=='dict':
            loc=self.eddy_t[self.keys_at_t[ii]]['position_default'][-1]
        else:
            pass
        return loc
    
    def list_locs(self):
        if self.type=='dict':
            locs_t=np.array([self.eddy_t1[key_t1]['position_default'][-1] for key_t1 in self.eddy_t1])
        else:
            pass
        return locs_t


In [7]:
eddy_T_T=eddyTimeTrack(new_dict_0,new_dict_1,'nearest')

In [8]:
new_dict=eddy_T_T.track_eddies()

In [9]:
eddy_T_T=eddyTimeTrack(new_dict,new_dict_2,'nearest')

In [10]:
new_dict1=eddy_T_T.track_eddies()

In [11]:
new_dict['eddyn_1']

{'2dgaussianfit': array([[ 0.68637703,  0.68544448,  0.46980229,  0.        ,  0.        ,
          0.        ]]),
 'angle': array([ 0.46980229]),
 'area': array([  1.03015935e+10,   2.80763488e+10,   5.94872234e+09]),
 'contour': [[array([ 308.625     ,  308.875     ,  309.05428646,  309.125     ,
           309.375     ,  309.47202944,  309.625     ,  309.65875498,
           309.71292567,  309.625     ,  309.375     ,  309.125     ,
           308.875     ,  308.83860142,  308.625     ,  308.51756043,
           308.4817361 ,  308.60416692,  308.625     ]),
   array([-42.14059211, -42.17318038, -42.125     , -42.10269471,
          -41.95491382, -41.875     , -41.66805609, -41.625     ,
          -41.375     , -41.27796871, -41.26637828, -41.29771303,
          -41.35849111, -41.375     , -41.50843198, -41.625     ,
          -41.875     , -42.125     , -42.14059211])]],
 'ellipse': [[array([ 309.67439612,  309.57748161,  309.41843856,  309.21644992,
           308.99587849,  308.7

In [12]:
new_dict1['eddyn_1']

{'2dgaussianfit': array([[ 0.68637703,  0.68544448,  0.46980229,  0.        ,  0.        ,
          0.        ]]),
 'angle': array([ 0.46980229]),
 'area': array([  1.03015935e+10,   2.80763488e+10,   5.94872234e+09]),
 'contour': [[array([ 308.625     ,  308.875     ,  309.05428646,  309.125     ,
           309.375     ,  309.47202944,  309.625     ,  309.65875498,
           309.71292567,  309.625     ,  309.375     ,  309.125     ,
           308.875     ,  308.83860142,  308.625     ,  308.51756043,
           308.4817361 ,  308.60416692,  308.625     ]),
   array([-42.14059211, -42.17318038, -42.125     , -42.10269471,
          -41.95491382, -41.875     , -41.66805609, -41.625     ,
          -41.375     , -41.27796871, -41.26637828, -41.29771303,
          -41.35849111, -41.375     , -41.50843198, -41.625     ,
          -41.875     , -42.125     , -42.14059211])]],
 'ellipse': [[array([ 309.67439612,  309.57748161,  309.41843856,  309.21644992,
           308.99587849,  308.7

In [13]:
count=0
for key,item in new_dict.items():
    count=count+len(item['time'])
print(count,len(new_dict_0.keys())+len(new_dict_1.keys()))

2138 2138


In [14]:
count=0
for key,item in new_dict1.items():
    count=count+len(item['time'])
print(count,len(new_dict_0.keys())+len(new_dict_1.keys())+len(new_dict_2.keys()))

3238 3238


In [16]:
# new_dict_0={}
# for key,item in eddyd.items():
    

In [17]:
eddyd['eddyn_0','eddyn_1']

KeyError: ('eddyn_0', 'eddyn_1')

In [19]:
np.hstack((1,2))

array([1, 2])

In [20]:
new_dict_0['eddyn_0']

{'2dgaussianfit': array([[ 0.68589054,  0.9501844 ,  0.88051107,  0.        ,  0.        ,
          0.        ]]),
 'angle': array([ 0.88051107]),
 'area': array([  2.59994461e+10,   2.92334897e+10,   2.82836143e+10]),
 'contour': [[array([ 18.875     ,  19.125     ,  19.375     ,  19.625     ,
           19.71609085,  19.875     ,  19.99467561,  20.10223643,
           20.125     ,  20.188266  ,  20.25553254,  20.14741263,
           20.125     ,  19.875     ,  19.67558016,  19.625     ,
           19.375     ,  19.37171456,  19.125     ,  19.04839065,
           18.875     ,  18.84234021,  18.71690731,  18.64523947,
           18.625     ,  18.59546524,  18.625     ,  18.69083322,  18.875     ]),
   array([-40.96427741, -41.00868001, -40.98838458, -40.92047245,
          -40.875     , -40.76763485, -40.625     , -40.375     ,
          -40.31028826, -40.125     , -39.875     , -39.625     ,
          -39.59843114, -39.43768744, -39.375     , -39.35005822,
          -39.37214517, -39

In [21]:
def track_from_file(filename):
    eddydict=np.load(filename)
    eddyd=eddydict.item()
    time_dict={}
    for key,item in eddyd.items():
        if int(item['time']) not in time_dict.keys():
            time_dict[int(item['time'])]={}
            time_dict[int(item['time'])][key]=item
        else:
            time_dict[int(item['time'])][key]=item
            
    for key in range(0,len(time_dict.keys())-1):
        print('time='+str(key))
        if key==0:
            eddy_T_T=eddyTimeTrack(time_dict[key],time_dict[key+1],'nearest')
            new_dict=eddy_T_T.track_eddies()
        else:
            eddy_T_T=eddyTimeTrack(new_dict,time_dict[key+1],'nearest')
            new_dict=eddy_T_T.track_eddies()
    
    return new_dict

In [22]:
filename='../data_bk/aviso_1993-1-3_pos.npy'

In [23]:
%%time
td=track_from_file(filename)

time=0
time=1
time=2
time=3
time=4
time=5
time=6
time=7
time=8
time=9
time=10
time=11
time=12
time=13
time=14
time=15
time=16
time=17
time=18
time=19
time=20
time=21
time=22
time=23
time=24
time=25
time=26
time=27
time=28
time=29
time=30
time=31
time=32
time=33
time=34
time=35
time=36
time=37
time=38
time=39
time=40
time=41
time=42
time=43
time=44
time=45
time=46
time=47
time=48
time=49
time=50
time=51
time=52
time=53
time=54
time=55
time=56
time=57
CPU times: user 1h 18min 53s, sys: 3min 28s, total: 1h 22min 22s
Wall time: 1h 22min 32s


In [29]:
a=0
for key,item in td.items():
    a=a+len(item['time'])
print(a)

63578


In [27]:
eddydict=np.load(filename)
eddyd=eddydict.item()
print(len(eddyd.keys()))

63578


# To do:
- make list of connected eddies, and then join them all.
